In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
from typing import List, Dict, Any, Tuple, Set
from multiprocessing import Pool, cpu_count
import itertools
from tqdm import tqdm

import os
import time
import torch

In [7]:
output_dir = 'D:/Thesis/files_output_dir/output_utilization/'

In [8]:
level1_road_history_workday_utility = pd.read_csv(output_dir + 'level1_road_history_workday_utilization_tune.csv')
level2_road_history_workday_utility = pd.read_csv(output_dir + 'level2_road_history_workday_utilization_tune.csv')

In [9]:
# Read the graph back from the GraphML file
G = nx.read_graphml(output_dir + "graph_1.graphml")
len(G.nodes())

21143

In [10]:
def graph_weight_update_road_utility(road_data, graph, time_slot=48):
    # Filter for the specific time slot
    time_slot_data = road_data[
        road_data['time'] == time_slot
    ]
    
    # Create a mapping of road_id to inv_utilization
    road_weights = dict(zip(
        time_slot_data['road_id'],
        time_slot_data['inv_utilization']
    ))
    
    # Update graph edge weights
    for u, v, data in graph.edges(data=True):
        road_id = data.get('road_id')
        if road_id in road_weights:
            graph[u][v]['weight'] = road_weights[road_id]
    
    return graph

In [11]:
road_history_workday_utility = pd.concat([
        level1_road_history_workday_utility[['time','road_id','utilization', 'inv_utilization']],
        level2_road_history_workday_utility[['time','road_id','utilization', 'inv_utilization']]
    ], ignore_index=True)

In [12]:
G_workday_48 = graph_weight_update_road_utility(road_history_workday_utility, G.copy(), time_slot=48)

In [ ]:
def identify_potential_origins_destinations(G):
    # Calculate node metrics
    degree_cent = nx.degree_centrality(G)
    betweenness_cent = nx.betweenness_centrality(G, weight='weight')
    
    # Get average metrics
    avg_degree = sum(degree_cent.values()) / len(degree_cent)
    avg_betweenness = sum(betweenness_cent.values()) / len(betweenness_cent)

    print('avg_degree:', avg_degree)
    print('avg_betweenness', avg_betweenness)
    
    # Identify potential origins (high degree, moderate-high betweenness)
    origins = [
        node for node in G.nodes()
        if (degree_cent[node] > avg_degree and 
            betweenness_cent[node] > avg_betweenness * 0.5)
    ]
    
    # Identify potential destinations (high betweenness, any degree)
    destinations = [
        node for node in G.nodes()
        if betweenness_cent[node] > avg_betweenness
    ]
    
    # Ensure minimum number of OD nodes
    min_od_nodes = min(50, len(G.nodes()) // 4)  # at least 50 nodes or 25% of network
    
    if len(origins) < min_od_nodes:
        # Add more nodes based on degree centrality
        additional_nodes = sorted(
            [(n, degree_cent[n]) for n in G.nodes() if n not in origins],
            key=lambda x: x[1],
            reverse=True
        )[:min_od_nodes - len(origins)]
        origins.extend([n for n, _ in additional_nodes])
    
    if len(destinations) < min_od_nodes:
        # Add more nodes based on betweenness centrality
        additional_nodes = sorted(
            [(n, betweenness_cent[n]) for n in G.nodes() if n not in destinations],
            key=lambda x: x[1],
            reverse=True
        )[:min_od_nodes - len(destinations)]
        destinations.extend([n for n, _ in additional_nodes])
    
    return list(set(origins)), list(set(destinations)), degree_cent, betweenness_cent

## Sequencial Processing

In [ ]:
def calculate_od_paths(G: nx.Graph, 
                      origins = None, 
                      destinations = None, 
                      max_pairs= 1000):
    
    if origins is None or destinations is None:
        origins, destinations = identify_potential_origins_destinations(G)
    
    # Initialize containers
    path_dataset = []
    processed_pairs = set()
    invalid_pairs = set()
    
    # Create pairs of origins and destinations
    all_pairs = [(o, d) for o in origins for d in destinations if o != d]
    
    # Log initial statistics
    print(f"Total potential OD pairs: {len(all_pairs)}")
    
    # Process pairs
    for o, d in all_pairs:
        # if len(path_dataset) >= max_pairs:
        #     print(f"Reached maximum number of pairs ({max_pairs})")
        #     break
            
        if (o, d) in processed_pairs or (o, d) in invalid_pairs:
            continue
            
        print(f"\nProcessing path from {o} to {d}:")
        
        # Check basic path existence
        if not nx.has_path(G, o, d):
            print(f"No path exists between {o} and {d}")
            invalid_pairs.add((o, d))
            continue
        
        try:
            # Find shortest path
            path = nx.shortest_path(G, o, d, weight='weight')
            
            # Validate path edges
            skip_path = False
            edge_path = []
            total_weight = 0
            
            # Check each edge in the path
            for i in range(len(path)-1):
                u, v = path[i], path[i+1]
                edge_data = G[u][v]
                
                # Check for valid weight
                weight = edge_data.get('weight')
                if weight is None or np.isnan(weight):
                    print(f"Edge ({u}, {v}) has invalid weight: {weight}")
                    skip_path = True
                    invalid_pairs.add((o, d))
                    break
                
                # Collect edge information
                edge_info = {
                    'edge': (u, v),
                    'weight': weight,
                    'road_id': edge_data.get('road_id'),
                    'length': len(path) - 1
                }
                edge_path.append(edge_info)
                total_weight += weight
            
            if not skip_path:
                # Create path entry
                path_entry = {
                    'origin': o,
                    'destination': d,
                    'path': edge_path,
                    'total_weight': total_weight,
                    'path_length': len(path) - 1,
                    'node_path': path
                }
                
                path_dataset.append(path_entry)
                processed_pairs.add((o, d))
                print(f"Valid path found with {len(path)-1} edges and total weight {total_weight:.3f}")
                
        except nx.NetworkXNoPath:
            print(f"No valid path exists between {o} and {d}")
            invalid_pairs.add((o, d))
            continue
        except Exception as e:
            print(f"Error processing path {o} to {d}: {str(e)}")
            invalid_pairs.add((o, d))
            continue
    
    # Final statistics
    print("\nPath Calculation Summary:")
    print(f"Total valid paths found: {len(path_dataset)}")
    print(f"Total invalid pairs: {len(invalid_pairs)}")
    print(f"Total processed pairs: {len(processed_pairs)}")
    
    # Basic validation statistics
    if path_dataset:
        weights = [path['total_weight'] for path in path_dataset]
        lengths = [path['path_length'] for path in path_dataset]
        print("\nPath Statistics:")
        print(f"Average path length: {np.mean(lengths):.2f} edges")
        print(f"Average path weight: {np.mean(weights):.2f}")
        print(f"Weight range: {np.min(weights):.2f} to {np.max(weights):.2f}")
    
    return path_dataset

################################################################################################

def analyze_paths(path_dataset):
    """
    Analyze the calculated paths
    """
    if not path_dataset:
        return {"error": "No paths in dataset"}
    
    path_stats = {
        'total_paths': len(path_dataset),
        'avg_path_length': sum(path['path'][0]['length'] for path in path_dataset) / len(path_dataset),
        'min_path_length': min(path['path'][0]['length'] for path in path_dataset),
        'max_path_length': max(path['path'][0]['length'] for path in path_dataset),
        'avg_total_weight': sum(path['total_weight'] for path in path_dataset) / len(path_dataset),
        'min_total_weight': min(path['total_weight'] for path in path_dataset),
        'max_total_weight': max(path['total_weight'] for path in path_dataset)
    }
    
    return path_stats

def format_path_for_printing(path_info):
    """
    Formats a path for readable printing
    """
    output = [
        f"Origin: {path_info['origin']}",
        f"Destination: {path_info['destination']}",
        "Path:",
    ]
    
    for edge_info in path_info['path']:
        edge = edge_info['edge']
        weight = edge_info['weight']
        road_id = edge_info['road_id']
        output.append(f"  Edge{edge}(road_{road_id}): {weight:.3f}")
    
    output.append(f"Total Weight: {path_info['total_weight']:.3f}")
    output.append(f"Path Length: {path_info['path'][0]['length']} edges")
    
    return "\n".join(output)

## Parallel Processing

In [13]:
def process_od_chunk(args):
    """
    Process a chunk of OD pairs in parallel.
    
    Parameters:
    -----------
    args : tuple
        (chunk_of_od_pairs, graph_data, chunk_id)
        
    Returns:
    --------
    list
        Valid paths found in this chunk
    """
    od_pairs, graph_data, chunk_id = args
    
    # Reconstruct graph from serialized data
    G = nx.Graph()
    G.add_edges_from(graph_data['edges'])
    nx.set_edge_attributes(G, graph_data['weights'], 'weight')
    nx.set_edge_attributes(G, graph_data['road_ids'], 'road_id')
    
    chunk_paths = []
    invalid_pairs = set()
    
    for o, d in od_pairs:
        try:
            if not nx.has_path(G, o, d):
                continue
                
            path = nx.shortest_path(G, o, d, weight='weight')
            
            # Validate path edges
            skip_path = False
            edge_path = []
            total_weight = 0
            
            for i in range(len(path)-1):
                u, v = path[i], path[i+1]
                edge_data = G[u][v]
                
                weight = edge_data.get('weight')
                if weight is None or np.isnan(weight):
                    skip_path = True
                    invalid_pairs.add((o, d))
                    break
                
                edge_info = {
                    'edge': (u, v),
                    'weight': weight,
                    'road_id': edge_data.get('road_id'),
                    'length': len(path) - 1
                }
                edge_path.append(edge_info)
                total_weight += weight
            
            if not skip_path:
                path_entry = {
                    'origin': o,
                    'destination': d,
                    'path': edge_path,
                    'total_weight': total_weight,
                    'path_length': len(path) - 1,
                    'node_path': path,
                    'chunk_id': chunk_id
                }
                chunk_paths.append(path_entry)
                
        except (nx.NetworkXNoPath, Exception):
            invalid_pairs.add((o, d))
            continue
    
    return chunk_paths
#---------------------------------------------------------------------------------------------------------------------
def save_chunk_to_disk(chunk_paths: List[Dict], chunk_id: int, output_dir: str):
    """Save chunk results to disk to free up memory"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    chunk_df = pd.DataFrame(chunk_paths)
    chunk_df.to_parquet(f"{output_dir}/chunk_{chunk_id}.parquet")

#---------------------------------------------------------------------------------------------------------------------------------------------
def calculate_od_paths_parallel(G: nx.Graph,
                              origins= None,
                              destinations = None,
                              max_pairs: int = None,
                              chunk_size: int = 10000,
                              n_cores: int = None,
                              output_dir: str = "od_path_chunks") -> List[Dict[str, Any]]:
    """
    Calculate shortest paths between OD pairs in parallel using chunking.
    
    Parameters:
    -----------
    G : networkx.Graph
        Network graph with edge weights
    origins : list, optional
        List of origin nodes
    destinations : list, optional
        List of destination nodes
    max_pairs : int, optional
        Maximum number of OD pairs to process
    chunk_size : int, default=10000
        Number of OD pairs to process in each chunk
    n_cores : int, optional
        Number of CPU cores to use
    output_dir : str, default="od_path_chunks"
        Directory to store chunk results
        
    Returns:
    --------
    list
        List of valid paths
    """
    # Set number of cores
    if n_cores is None:
        n_cores = min(10, cpu_count())  # Use max 16 cores or available cores
    
    print(f"Using {n_cores} CPU cores")
    
    # Create pairs of origins and destinations
    all_pairs = list(itertools.product(origins, destinations))
    all_pairs = [(o, d) for o, d in all_pairs if o != d]
    
    if max_pairs:
        all_pairs = all_pairs[:max_pairs]
    
    total_pairs = len(all_pairs)
    print(f"Total OD pairs to process: {total_pairs}")
    
    # Prepare graph data for serialization
    graph_data = {
        'edges': list(G.edges()),
        'weights': nx.get_edge_attributes(G, 'weight'),
        'road_ids': nx.get_edge_attributes(G, 'road_id')
    }
    
    # Create chunks
    chunks = []
    for i in range(0, len(all_pairs), chunk_size):
        chunk = all_pairs[i:i + chunk_size]
        chunks.append((chunk, graph_data, i//chunk_size))
    
    print(f"Created {len(chunks)} chunks of size {chunk_size}")
    
    # Process chunks in parallel
    start_time = time.time()
    path_dataset = []
    
    with Pool(n_cores) as pool:
        # Process chunks with progress bar
        for chunk_paths in tqdm(pool.imap_unordered(process_od_chunk, chunks),
                              total=len(chunks),
                              desc="Processing chunks"):
            # Save chunk results to disk
            if chunk_paths:
                save_chunk_to_disk(chunk_paths, len(path_dataset), output_dir)
                path_dataset.extend(chunk_paths)
    
    end_time = time.time()
    print(f"\nProcessing completed in {end_time - start_time:.2f} seconds")
    
    # Compile statistics
    print("\nPath Calculation Summary:")
    print(f"Total valid paths found: {len(path_dataset)}")
    print(f"Success rate: {len(path_dataset)/total_pairs*100:.2f}%")
    
    if path_dataset:
        weights = [path['total_weight'] for path in path_dataset]
        lengths = [path['path_length'] for path in path_dataset]
        print("\nPath Statistics:")
        print(f"Average path length: {np.mean(lengths):.2f} edges")
        print(f"Average path weight: {np.mean(weights):.2f}")
        print(f"Weight range: {np.min(weights):.2f} to {np.max(weights):.2f}")
    
    return path_dataset

#--------------------------------------------------------------------------------------------------------------
def combine_chunk_results(output_dir: str) -> pd.DataFrame:
    """
    Combine all chunk results into a single DataFrame
    """
    all_chunks = []
    for chunk_file in os.listdir(output_dir):
        if chunk_file.endswith('.parquet'):
            chunk_df = pd.read_parquet(os.path.join(output_dir, chunk_file))
            all_chunks.append(chunk_df)
    
    return pd.concat(all_chunks, ignore_index=True)

#example usage
'''
# Set parameters
chunk_size = 10000  # Adjust based on available memory
n_cores = 10  # Use 10 cores
output_chunk_dir = "od_path_results"

# Calculate paths in parallel
od_path_dataset = calculate_od_paths_parallel(
    G_workday_48,
    origins= source_hubs,
    destinations=destination_hubs,
    chunk_size=chunk_size,
    n_cores=n_cores,
    output_dir=output_chunk_dir
)
'''


'\n# Set parameters\nchunk_size = 10000  # Adjust based on available memory\nn_cores = 10  # Use 10 cores\noutput_chunk_dir = "od_path_results"\n\n# Calculate paths in parallel\nod_path_dataset = calculate_od_paths_parallel(\n    G_workday_48,\n    origins= source_hubs,\n    destinations=destination_hubs,\n    chunk_size=chunk_size,\n    n_cores=n_cores,\n    output_dir=output_chunk_dir\n)\n'

In [ ]:
# Calculate paths using graph structure to identify OD pairs
# od_path_dataset = calculate_od_paths(G_workday_48)

# origins, destinations, degree_cent, betweenness_cent = identify_potential_origins_destinations(G)

In [ ]:
# # Create pairs of origins and destinations
# all_pairs = [(o, d) for o in origins for d in destinations if o != d]

# # Log initial statistics
# print(f"Total potential OD pairs: {len(all_pairs)}")

Total potential OD pairs: 21804876


In [14]:
 # Analyze node degrees
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())
k =500

# Find important nodes

# Nodes with highest in-degree (destination hubs)
high_in_degree = sorted(in_degrees.items(), key=lambda x: x[1], reverse=True)[:k]
destination_hubs =[int(node) for node, degree in high_in_degree]

# Nodes with highest out-degree (source hubs)
high_out_degree = sorted(out_degrees.items(), key=lambda x: x[1], reverse=True)[:k]
source_hubs = [int(node) for node, degree in high_out_degree]

# Create pairs of origins and destinations
all_pairs = [(o, d) for o in source_hubs for d in destination_hubs if o != d]

# Log initial statistics
print(f"Total potential OD pairs: {len(all_pairs)}")

Total potential OD pairs: 249833


In [ ]:
# Set parameters
chunk_size = 10000  # Adjust based on available memory
n_cores = 10  # Use 10 cores
output_chunk_dir = "od_path_results"

# Calculate paths in parallel
od_path_dataset = calculate_od_paths_parallel(
    G_workday_48,
    origins= source_hubs,
    destinations=destination_hubs,
    chunk_size=chunk_size,
    n_cores=n_cores,
    output_dir=output_chunk_dir
)

# # Combine results if needed
# full_results_df = combine_chunk_results(output_chunk_dir)

# # Clean up temporary files if desired
# import shutil
# shutil.rmtree(output_chunk_dir)


Using 10 CPU cores
Total OD pairs to process: 249833
Created 25 chunks of size 10000


Processing chunks:   0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
# od_path_dataset = calculate_od_paths(G_workday_48, origins, destinations)

# # Analyze the paths
# stats = analyze_paths(od_path_dataset)
# print("\nPath Statistics:")
# for key, value in stats.items():
#     print(f"{key}: {value}")

# Print some example paths
# print("\nExample Paths:")
# for path_info in od_path_dataset[:5]:  # print first 5 paths
#     print("\n" + format_path_for_printing(path_info))


Total potential OD pairs: 22116719

Processing path from 4845 to 54055:
Edge (52041, 53536) has invalid weight: nan

Processing path from 4845 to 56790:
Edge (68661, 69440) has invalid weight: nan

Processing path from 4845 to 67359:
Edge (65991, 65989) has invalid weight: nan

Processing path from 4845 to 10182:
Valid path found with 81 edges and total weight 175.971

Processing path from 4845 to 20060:
Valid path found with 56 edges and total weight 106.817

Processing path from 4845 to 4850:
Valid path found with 9 edges and total weight 19.497

Processing path from 4845 to 6117:
Edge (2359, 745) has invalid weight: nan

Processing path from 4845 to 20497:
Valid path found with 219 edges and total weight 399.745

Processing path from 4845 to 44603:
Valid path found with 109 edges and total weight 207.271

Processing path from 4845 to 7572:
Edge (7175, 7180) has invalid weight: nan

Processing path from 4845 to 40177:
Valid path found with 206 edges and total weight 426.774

Processi

KeyboardInterrupt: 

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

def visualize_od_paths(G, path_dataset, figsize=(15, 10), node_size=100, 
                      edge_width_scale=2, title="Network OD Paths Visualization"):
    """
    Visualize the network with OD paths highlighted based on their weights.
    
    Parameters:
    -----------
    G : networkx.Graph
        Original network graph
    path_dataset : list
        List of paths with weights from calculate_od_paths function
    figsize : tuple, default=(15, 10)
        Figure size
    node_size : int, default=100
        Size of nodes in visualization
    edge_width_scale : float, default=2
        Scaling factor for edge widths
    title : str, default="Network OD Paths Visualization"
        Title of the plot
    """
    # Create a new figure
    plt.figure(figsize=figsize)
    
    # Create a copy of the graph for visualization
    G_viz = G.copy()
    
    # Calculate edge frequencies and total weights
    edge_weights = {}
    edge_frequencies = {}
    
    # Collect all weights for normalization
    all_weights = []
    
    # Process each path
    for path_info in path_dataset:
        for edge_info in path_info['path']:
            edge = edge_info['edge']
            weight = edge_info['weight']
            
            # Update edge frequencies
            edge_frequencies[edge] = edge_frequencies.get(edge, 0) + 1
            
            # Update cumulative weights
            if edge not in edge_weights:
                edge_weights[edge] = weight
            else:
                edge_weights[edge] = max(edge_weights[edge], weight)
            
            all_weights.append(weight)
    
    # Normalize weights for visualization
    max_weight = max(all_weights)
    min_weight = min(all_weights)
    weight_range = max_weight - min_weight
    
    # Create positions for nodes (you might want to adjust the layout algorithm)
    pos = nx.spring_layout(G_viz, k=1/np.sqrt(len(G_viz.nodes())), iterations=50)
    
    # Draw the base network (edges not in paths)
    nx.draw_networkx_edges(G_viz, pos, 
                          edgelist=[(u, v) for (u, v) in G_viz.edges() if (u, v) not in edge_weights],
                          edge_color='lightgray',
                          width=0.5,
                          alpha=0.3)
    
    # Create custom colormap
    colors = ['lightblue', 'blue', 'darkblue']
    n_bins = 100
    cmap = LinearSegmentedColormap.from_list("custom", colors, N=n_bins)
    
    # Draw edges in paths with varying width and color
    for edge, weight in edge_weights.items():
        # Normalize weight for color mapping
        norm_weight = (weight - min_weight) / weight_range if weight_range > 0 else 0.5
        
        # Calculate edge width based on frequency
        width = np.log1p(edge_frequencies[edge]) * edge_width_scale
        
        # Draw the edge
        nx.draw_networkx_edges(G_viz, pos,
                             edgelist=[edge],
                             edge_color=[norm_weight],
                             edge_cmap=cmap,
                             width=width)
    
    # Identify origins and destinations
    origins = set(path_info['origin'] for path_info in path_dataset)
    destinations = set(path_info['destination'] for path_info in path_dataset)
    
    # Draw different types of nodes
    # Regular nodes
    regular_nodes = [node for node in G_viz.nodes() 
                    if node not in origins and node not in destinations]
    if regular_nodes:
        nx.draw_networkx_nodes(G_viz, pos, 
                             nodelist=regular_nodes,
                             node_color='lightgray',
                             node_size=node_size)
    
    # Origin nodes
    if origins:
        nx.draw_networkx_nodes(G_viz, pos, 
                             nodelist=list(origins),
                             node_color='green',
                             node_size=node_size*1.5)
    
    # Destination nodes
    if destinations:
        nx.draw_networkx_nodes(G_viz, pos, 
                             nodelist=list(destinations),
                             node_color='red',
                             node_size=node_size*1.5)
    
    # Add labels for important nodes
    important_nodes = origins.union(destinations)
    labels = {node: str(node) for node in important_nodes}
    nx.draw_networkx_labels(G_viz, pos, labels, font_size=8)
    
    # Add a colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap)
    sm.set_array([])
    plt.colorbar(sm, label='Edge Weight')
    
    # Add legend
    plt.plot([], [], 'go', label='Origins', markersize=8)
    plt.plot([], [], 'ro', label='Destinations', markersize=8)
    plt.legend()
    
    plt.title(title)
    plt.axis('off')
    
    # Add text with statistics
    stats_text = (
        f"Total Paths: {len(path_dataset)}\n"
        f"Origins: {len(origins)}\n"
        f"Destinations: {len(destinations)}\n"
        f"Active Edges: {len(edge_weights)}"
    )
    plt.text(0.02, 0.98, stats_text, 
             transform=plt.gca().transAxes,
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    
    return plt.gcf()

# Example usage:
"""
# Calculate paths
od_path_dataset = calculate_od_paths(G_workday_48)

# Create visualization
fig = visualize_od_paths(G_workday_48, od_path_dataset)
plt.show()

# Save the figure if needed
# fig.savefig('network_od_paths.png', dpi=300, bbox_inches='tight')
"""

def create_interactive_network_visualization(G, path_dataset):
    """
    Create an interactive HTML visualization of the network using Pyvis
    Useful for exploring large networks
    """
    from pyvis.network import Network
    
    # Create a new network
    net = Network(notebook=True, height="750px", width="100%")
    
    # Calculate edge weights and frequencies
    edge_weights = {}
    edge_frequencies = {}
    for path_info in path_dataset:
        for edge_info in path_info['path']:
            edge = edge_info['edge']
            weight = edge_info['weight']
            edge_frequencies[edge] = edge_frequencies.get(edge, 0) + 1
            if edge not in edge_weights:
                edge_weights[edge] = weight
            else:
                edge_weights[edge] = max(edge_weights[edge], weight)
    
    # Add nodes
    origins = set(path_info['origin'] for path_info in path_dataset)
    destinations = set(path_info['destination'] for path_info in path_dataset)
    
    for node in G.nodes():
        color = 'lightgray'
        if node in origins:
            color = 'green'
        elif node in destinations:
            color = 'red'
        
        net.add_node(node, label=str(node), color=color)
    
    # Add edges
    max_weight = max(edge_weights.values())
    for (u, v) in G.edges():
        if (u, v) in edge_weights:
            weight = edge_weights[(u, v)]
            frequency = edge_frequencies[(u, v)]
            width = np.log1p(frequency) * 2
            value = weight / max_weight  # normalized weight for color intensity
            
            net.add_edge(u, v, width=width, value=value, title=f"Weight: {weight:.2f}")
        else:
            net.add_edge(u, v, color='lightgray', width=0.5)
    
    # Set physics layout options
    net.toggle_physics(True)
    net.show_buttons(filter_=['physics'])
    
    return net

# Example usage for interactive visualization:
"""
# Create interactive network
net = create_interactive_network_visualization(G_workday_48, od_path_dataset)
net.show('network_visualization.html')
"""